(ch_conv)=
# 卷积

卷积（convolution，简称 *CONV*）算子是神经网络中最昂贵、最流行的算子之一。在本节中，将介绍具有单个输入和输出通道的算子。请参阅 D2L 的 [6.2](https://d2l.ai/chapter_convolutional-neural-networks/conv-layer.html)、[6.3](http://d2l.ai/chapter_convolutional-neural-networks/padding-and-strides.html) 和 [6.4](http://d2l.ai/chapter_convolutional-neural-networks/channels.html) 章，了解更多关于该算子的说明。在这里，不会解释太多与卷积相关的术语，如填充、通道、步幅、卷积核等。

In [ ]:
%cd ../..
import set_env

In [2]:
import numpy as np
import tvm
from tvm import te

## Padding

作为卷积的先决条件，首先实现填充，它在视觉上用 "shell" 包围目标张量。填充值通常为 0。

```{note}
在 {ref}`ch_all_any` 引入 `te.any` 时，简单地触及了的填充，这是二维矩阵的填充。在此，将填充推广到 $n$-D 张量的二维张量卷积中，它通常用于神经网络的卷积算子。
```

在一般情况下，假设最后两个维度是行和列，0 只在这两个维度上填充。特别是，如果矩阵高度（即行数） $n_h$ 和宽度（即列数）是 $n_w$，然后在顶部和底部将填充 $p_h$ 行，在左右各填充 $p_w$，使其高度为 $n_h+2p_h$ 和宽度为 $n_w+2p_w$。已经在 {ref}`ch_shapes` 中提到过一次，但要再次注意，`te.compute` 中的 `*X` 和 `*i` 是用来表示一般的多维张量的。

In [3]:
def padding(X, ph, pw, val=0):
    """Pad X with the given value in 2-D

    ph, pw : height and width padding
    val : padding value, default 0
    """
    assert len(X.shape) >= 2
    nh, nw = X.shape[-2], X.shape[-1]
    return te.compute(
            (*X.shape[0:-2], nh+ph*2, nw+pw*2),
            lambda *i: te.if_then_else(
                te.any(i[-2]<ph, i[-2]>=nh+ph, i[-1]<pw, i[-1]>=nw+pw),
                val, X[i[:-2]+(i[-2]-ph, i[-1]-pw)]),
            name='PaddedX')

验证三维张量的结果。

In [4]:
A = te.placeholder((2, 3, 4))
B = padding(A, 1, 2)
s = te.create_schedule(B.op)
mod = tvm.build(s, [A, B])

a = tvm.nd.array(np.ones((2, 3, 4), dtype='float32'))
b = tvm.nd.array(np.empty((2, 5, 8), dtype='float32'))
mod(a, b)
print(b)

[[[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]]


## 卷积实现

首先考虑简单的单通道卷积。给定 $n_h\times n_w$ 数据矩阵 $X$，首先填充 0 到 $(n_h+2p_h) \times (n_w+2p_w)$。若核矩阵 $K$ 尺寸为 $k_h\times k_w$，使用 stride 的高度 $s_h$ 和宽度 $s_w$，输出 $Y = X \star K$  将有形状

$$ \lfloor (n_h-k_h+2p_h)/s_h+1\rfloor  \times \lfloor (n_w-k_w+2p_w)/s_w+1\rfloor.$$

并且 $Y$ 的元素可以通过计算 $Y_{i,j}$

$$ Y_{i,j} = \sum_{a=0}^{k_w-1}\sum_{b=0}^{k_h-1} X_{i s_w+a, j s_h+b} K_{a, b}$$

```{figure} ../img/conv-stride.svg
2-D 卷积，填充值为1，步长分别为 3 和 2，对应高度和宽度。阴影部分描述了两个输出元素，以及用于计算它们的相应输入和卷积核数组元素 $0\times0+0\times1+1\times2+2\times3=8$, $0\times0+6\times1+0\times2+0\times3=6$。
```

现在看一个更一般的多通道情况。

假设有 $c_i \times n_h \times n_w$ 输入张量 $X$，和 $c_o\times c_i\times k_h\times k_w$ 4-D 核张量 $K$，这里 $c_i$ 和 $c_o$ 分别是输入通道和输出通道的数目。然后输出 $Y$ 有形状

$$ c_o \times \lfloor (h-k_h+2p_h)/s_h+1\rfloor  \times \lfloor (w-k_w+2p_w)/s_w+1\rfloor.$$

特别地，第 $i$ 个二维矩阵 $Y_i$，$i=1,\ldots,c_o$，定义如下

$$ Y_i = \sum_{j=1}^n X_j \star K_{i,j},$$


这里 $K_{i,j}$ 是输出通道 $i$ 和输入通道 $j$ 的 2-D 卷积核矩阵。

在深度学习 workload 中，特别是在训练中，经常将多个输入连接到批处理中。一批输入具有形状 $n \times c_i \times n_h \times n_w$，其中 $n$ 是批大小。将卷积应用到批处理中意味着分别对 3-D 张量进行卷积，然后将结果拼接成 4-D 张量，其第一个维的尺寸是 $n$。

请注意，这里使用的输入布局称为 `NCHW`，这意味着输入张量的 4 个维度分别是批大小、通道、高度和宽度。通常，`NCHW` 意味着数据被安排在内存中，`N `是最外层的维数，`W` 是最内部的维数。有时使用其他数据布局，如 `NHWC`，它可以提供更高的性能。稍后将详细讨论这个问题。同样，卷积核布局定义为 `KCRS`，对应于输出通道、输入通道、卷积核高度和宽度。

在实现卷积之前，定义了一种给定输入宽度或高度来计算输出宽度或高度的方法。

In [5]:
# Save to the d2ltvm package.
def conv_out_size(n, k, p, s):
    """Compute the output size by given input size n (width or height),
    kernel size k, padding p, and stride s
    Return output size (width or height)
    """
    return (n - k + 2 * p)//s + 1

现在实现卷积。为简单起见，只考虑单个批处理的情况，即 `N=1`。在这种情况下，输入数据布局可以被视为 `CHW`。

In [6]:
# Save to the d2ltvm package.
def conv(oc, ic, nh, nw, kh, kw, ph=0, pw=0, sh=1, sw=1):
    """Convolution

    oc, ic : output and input channels
    nh, nw : input width and height
    kh, kw : kernel width and height
    ph, pw : height and width padding sizes, default 0
    sh, sw : height and width strides, default 1
    """
    # reduction axes
    ric = te.reduce_axis((0, ic), name='ric')
    rkh = te.reduce_axis((0, kh), name='rkh')
    rkw = te.reduce_axis((0, kw), name='rkw')
    # output height and width
    oh = conv_out_size(nh, kh, ph, sh)
    ow = conv_out_size(nw, kw, pw, sw)
    # pad X and then compute Y
    X = te.placeholder((ic, nh, nw), name='X')
    K = te.placeholder((oc, ic, kh, kw), name='K')
    PaddedX = padding(X, ph, pw) if ph * pw != 0 else X
    Y = te.compute(
        (oc, oh, ow),
        lambda c, i, j: te.sum(
            PaddedX[ric, i*sh+rkh, j*sw+rkw] * K[c, ric, rkh, rkw],
            axis=[ric, rkh, rkw]), name='Y')
    return X, K, Y, PaddedX

正如在 {ref}`ch_vector_add_te` 中创建的 `get_abc` 一样，定义方法来获取输入和输出张量。同样，固定随机种子，使它在多次调用时返回相同的结果。

In [7]:
def get_conv_data(oc, ic, n, k, p=0, s=1, constructor=None):
    """Return random 3-D data tensor, 3-D kernel tenor and empty 3-D output 
    tensor with the shapes specified by input arguments.

    oc, ic : output and input channels
    n : input width and height
    k : kernel width and height
    p : padding size, default 0
    s : stride, default 1
    constructor : user-defined tensor constructor
    """
    np.random.seed(0)
    data = np.random.normal(size=(ic, n, n)).astype('float32')
    weight = np.random.normal(size=(oc, ic, k, k)).astype('float32')
    on = conv_out_size(n, k, p, s)
    out = np.empty((oc, on, on), dtype='float32')
    if constructor:
        data, weight, out = (constructor(x) for x in [data, weight, out])
    return data, weight, out

现在编译模块并计算结果。

In [8]:
oc, ic, n, k, p, s = 4, 6, 12, 3, 1, 1
X, K, Y, _ = conv(oc, ic, n, n, k, k, p, p, s, s)
sch = te.create_schedule(Y.op)

m = tvm.lower(sch, [X, K, Y], simple_mode=True)

data, weight, out = get_conv_data(oc, ic, n, k, p, s, tvm.nd.array)
mod = tvm.build(m)
mod(data, weight, out)
m["main"]

PrimFunc([X, K, Y]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  allocate PaddedX[float32 * 1176], storage_scope = global
  for (i0, 0, 6) {
    for (i1, 0, 14) {
      for (i2, 0, 14) {
        PaddedX[(((i0*196) + (i1*14)) + i2)] = tir.if_then_else(((((i1 < 1) || (i1 == 13)) || (i2 < 1)) || (i2 == 13)), 0f, X[((((i0*144) + (i1*12)) + i2) - 13)])
      }
    }
  }
  for (c, 0, 4) {
    for (i, 0, 12) {
      for (j, 0, 12) {
        Y[(((c*144) + (i*12)) + j)] = 0f
        for (ric, 0, 6) {
          for (rkh, 0, 3) {
            for (rkw, 0, 3) {
              let cse_var_1 = (((c*144) + (i*12)) + j)
              Y[cse_var_1] = (Y[cse_var_1] + (PaddedX[(((((ric*196) + (i*14)) + (rkh*14)) + j) + rkw)]*K[((((c*54) + (ric*9)) + (rkh*3)) + rkw)]))
            }
          }
        }
      }
    }
  }
}

在最后的代码块中，还打印了 2-D 卷积的伪代码，这是朴素的 6 级嵌套 for 循环。

由于 NumPy 只对向量进行卷积，所以使用 MXNet 的卷积算子作为基础真值。下面的代码块定义了数据生成函数和调用卷积运算符的 wrap 函数。然后可以输入相同的张量，在 MXNet 中计算结果。

In [9]:
import mxnet as mx

def get_conv_data_mxnet(oc, ic, n, k, p, s, ctx='cpu'):
    ctx = getattr(mx, ctx)()
    data, weight, out = get_conv_data(oc, ic, n, k, p, s,
                                      lambda x: mx.nd.array(x, ctx=ctx))
    data, out = data.expand_dims(axis=0), out.expand_dims(axis=0)
    bias = mx.nd.zeros(out.shape[1], ctx=ctx)
    return data, weight, bias, out

# Save to the d2ltvm package.
def conv_mxnet(data, weight, bias, out, k, p, s):
    mx.nd.Convolution(data, weight, bias, kernel=(k,k), stride=(s,s),
                      pad=(p,p), num_filter=out.shape[1], out=out)

data, weight, bias, out_mx = get_conv_data_mxnet(oc, ic, n, k, p, s)
conv_mxnet(data, weight, bias, out_mx, k, p, s)

最后，比较一下结果。由于上一章提到的类似原因，在 MXNet 中使用的多线程使在这里使用了相对较大的容错。

In [10]:
np.testing.assert_allclose(out_mx[0].asnumpy(), out.asnumpy(), atol=1e-5)

## 小结

- 可以用一种相当简单的方式来表示 TVM 中二维卷积的计算。
— 深度学习工作负载通常是在 4-D 数据张量和卷积核张量上进行二维卷积。
- 简单的 2-D 卷积是 6 层嵌套的 for 循环。